In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import acquire as a
import prepare_regression as pr
import wrangle as w
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

- As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

- In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

- Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.
- Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.
- Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [2]:
df_zillow = a.get_zillow_data()


In [3]:
df_zillow.shape

(2152863, 7)

Summarize and clean

In [4]:
df_zillow.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 131.4 MB


In [5]:
df_zillow.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.152852e+06,2.152852e+06,2.144379e+06,2.152370e+06,2.143526e+06,2.148421e+06,2.152863e+06
mean,3.287196e+00,2.230688e+00,1.862855e+03,4.618962e+05,1.960950e+03,5.634866e+03,6.048377e+03
std,9.547544e-01,9.992796e-01,1.222125e+03,6.996760e+05,2.216220e+01,8.178910e+03,2.043329e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881702e+05,1.949000e+03,2.534980e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108950e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414320e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03


In [6]:
df_zillow.isna().any()

bedroomcnt                       True
bathroomcnt                      True
calculatedfinishedsquarefeet     True
taxvaluedollarcnt                True
yearbuilt                        True
taxamount                        True
fips                            False
dtype: bool

In [7]:
df_zillow.isna().sum()
#Alot of nulls in 3 columns, have to decide what to do with them

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

- renamed columns to make them easier to read
- set upper limits for bathrooms and bedrooms to less than or equal to 8
- converted data type of sqft, bedroom, and year_built to integer
- replaced values in county (formerly fips) column to match name of county for the given 4 digit code
- dropped duplicates
- stored cleaning and train, test, split in one function named prep_zillow that returns 3 clean dataframes (train, validate, test)

In [8]:

def prep_zillow(df):
    #rename columns
    df = df.rename(columns={'bedroomcnt':'bedrooms', 
                          'bathroomcnt':'bathrooms',
                          'yearbuilt':'year_built',
                          'calculatedfinishedsquarefeet':'total_sqft',
                          'fips': 'county',
                          'taxvaluedollarcnt': 'tax_value'
                          })
    #set upper and lower limits for bathrooms and bedrooms
    df = df[df.bathrooms <= 8]
    df = df[df.bathrooms >= 1]
    df = df[df.bedrooms <= 8]
    df = df[df.bedrooms >= 2]
    
     #convert data types 
    
    df.total_sqft = df.total_sqft.convert_dtypes(int)
    df.bedrooms = df.bedrooms.convert_dtypes(int)
    df.year_built = df.year_built.convert_dtypes(int)

     #replace 4 digit fips code with name of county
    df.county = df.county.replace(6059.0,'Orange').replace(6037.0,'Los_Angeles').replace(6111.0,'Ventura')
    

    df.drop_duplicates(inplace=True)
    
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123)
    
    return train, validate, test

In [9]:
#calling function to show output
train, validate, test = w.prep_zillow(df_zillow)

In [10]:
train.shape, validate.shape, test.shape

((1178309, 7), (504990, 7), (420825, 7))

In [11]:
train.head()

,bedrooms,bathrooms,total_sqft,tax_value,year_built,taxamount,county
442389,4,2.0,1584,222541.0,1956,2720.06,Los_Angeles
410878,3,1.0,1070,156253.0,1951,2051.36,Los_Angeles
1797787,4,3.0,2859,554783.0,2004,7953.54,Los_Angeles
351902,4,3.0,1859,143833.0,1976,2262.81,Los_Angeles
808866,3,2.0,2188,468470.0,1959,5882.29,Los_Angeles
